## Data Introduction

In [1]:
import pandas as pd
from tqdm import tqdm

In [2]:
jobs = pd.read_csv("../data/fullsample.csv")

jobs = jobs[jobs['END'] != 'Unknown']
jobs = jobs[jobs['STATE'] == 'COMPLETED']
jobs['BEGIN'] = pd.to_datetime(jobs['BEGIN'])
jobs['END'] = pd.to_datetime(jobs['END'])
jobs['REQTIME'] = pd.to_timedelta(jobs['REQTIME'])
jobs['USEDTIME'] = pd.to_timedelta(jobs['USEDTIME'])

jobs.head(5)

,JOBID,STATE,BEGIN,END,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE
1,30853133,COMPLETED,2021-08-06 11:36:09,2021-09-05 11:36:32,262144Mn,20604.62M,-125 days +00:00:00,-126 days +23:59:37,1,1,cgw-platypus,0:0
2,30858137,COMPLETED,2021-08-06 19:04:39,2021-09-05 19:04:53,204800Mn,57553.77M,-125 days +00:00:00,-126 days +23:59:46,1,32,cgw-tbi01,0:0
3,30935078,COMPLETED,2021-08-09 16:52:51,2021-09-07 20:52:55,65536Mn,20577.96M,-121 days +00:00:00,-122 days +23:59:56,1,8,cgw-platypus,0:0
4,31364111_2,COMPLETED,2021-08-17 07:45:07,2021-09-10 16:45:24,16384Mn,9733.43M,-101 days +15:00:00,-101 days +14:59:43,1,1,production,0:0
5,31364111_3,COMPLETED,2021-08-17 07:45:07,2021-09-06 16:17:34,16384Mn,9708.04M,-101 days +15:00:00,-84 days +07:27:33,1,1,production,0:0


In [3]:
jobs['END'].value_counts()

END
2020-12-18 09:41:23    559
2021-10-02 22:48:56    551
2021-07-12 11:36:02    312
2021-10-03 13:36:05    311
2021-02-17 16:45:58    297
                      ... 
2021-06-26 12:51:45      1
2021-06-26 12:52:59      1
2021-06-26 12:53:50      1
2021-06-26 12:53:54      1
2020-10-31 23:49:43      1
Name: count, Length: 4109550, dtype: int64

The fullsample dataset contains job records, with one row per job.

Each job gets a unique ID, contained in the **JOBID** column.

Some jobs can be submitted as arrays of similar jobs. These are listed with an underscore in the JOBID, where the number after the underscore indicates the tasknumber. For example. JOBID 31781951 was an array job with 10 parts. 

In [4]:
jobs[jobs['JOBID'].str.contains('31781951')]

,JOBID,STATE,BEGIN,END,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE
533,31781951_1,COMPLETED,2021-08-30 12:51:30,2021-09-08 02:17:41,16384Mn,10234.37M,-50 days,-34 days +02:33:49,1,12,production,0:0
534,31781951_2,COMPLETED,2021-08-30 12:51:30,2021-09-07 18:04:48,16384Mn,10247.40M,-50 days,-34 days +10:46:42,1,12,production,0:0
535,31781951_3,COMPLETED,2021-08-31 09:14:29,2021-09-08 16:36:06,16384Mn,10064.47M,-50 days,-34 days +08:38:23,1,12,production,0:0
536,31781951_4,COMPLETED,2021-09-01 01:59:50,2021-09-08 08:48:28,16384Mn,10004.80M,-50 days,-30 days +13:11:22,1,12,production,0:0
537,31781951_5,COMPLETED,2021-09-02 00:09:27,2021-09-08 23:58:57,16384Mn,9858.72M,-50 days,-26 days +00:10:30,1,12,production,0:0
538,31781951_6,COMPLETED,2021-09-02 16:19:55,2021-09-10 11:16:57,16384Mn,10065.06M,-50 days,-30 days +01:02:58,1,12,production,0:0
539,31781951_7,COMPLETED,2021-09-02 22:26:08,2021-09-10 18:48:31,16384Mn,10092.55M,-50 days,-31 days +23:37:37,1,12,production,0:0
540,31781951_8,COMPLETED,2021-09-03 10:54:14,2021-09-11 09:32:28,16384Mn,10146.98M,-50 days,-31 days +21:21:46,1,12,production,0:0
541,31781951_9,COMPLETED,2021-09-04 22:54:03,2021-09-12 16:16:04,16384Mn,10050.81M,-50 days,-30 days +02:37:59,1,12,production,0:0
542,31781951_10,COMPLETED,2021-09-06 06:54:35,2021-09-14 13:02:37,16384Mn,10042.53M,-50 days,-34 days +09:51:58,1,12,production,0:0


Jobs can have a few differents states, with the most common one being 'COMPLETED'. 

In [5]:
jobs['STATE'].value_counts()

STATE
COMPLETED              7375084
CANCELLED                 9055
FAILED                    3766
CANCELLED by 9201         1776
OUT_OF_MEMORY             1739
                        ...   
CANCELLED by 777651          1
CANCELLED by 761544          1
CANCELLED by 793827          1
CANCELLED by 651701          1
CANCELLED by 890137          1
Name: count, Length: 143, dtype: int64

The **BEGIN** field indicates when the job was started (initiated on a computer node).

The **END** field indicates when the job ended (completed, failed, or was cancelled while running).

The **REQMEM** field is the amount of memory requested in megabytes. It can be per-core/CPU (Mc) or per-node (Mn).



In [6]:
# Jobs where memory was requested per core.
jobs[jobs['REQMEM'].str[-2:] == 'Mc'].head()

,JOBID,STATE,BEGIN,END,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE
501,31776583_1,COMPLETED,2021-08-30 10:16:59,2021-09-01 02:04:11,4096Mc,1792.43M,-59 days +16:00:00,-5 days +04:12:48,1,1,production,0:0
502,31776584_12,COMPLETED,2021-08-30 10:17:00,2021-09-01 00:20:15,4096Mc,1792.43M,-59 days +16:00:00,-5 days +05:56:45,1,1,production,0:0
915,31793401_958,COMPLETED,2021-08-31 19:36:46,2021-09-01 00:37:11,4096Mc,2788.05M,0 days 05:00:00,0 days 05:00:25,1,1,production,0:0
916,31793401_987,COMPLETED,2021-08-31 20:33:46,2021-09-01 00:02:57,4096Mc,2779.27M,0 days 05:00:00,0 days 03:29:11,1,1,production,0:0
4727,31813223_1296,COMPLETED,2021-08-31 19:42:46,2021-09-01 00:43:15,4096Mc,2786.44M,0 days 05:00:00,0 days 05:00:29,1,1,production,0:0


In [7]:
# Jobs where memory was requested per node.
jobs[jobs['REQMEM'].str[-2:] == 'Mn'].head()

,JOBID,STATE,BEGIN,END,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE
1,30853133,COMPLETED,2021-08-06 11:36:09,2021-09-05 11:36:32,262144Mn,20604.62M,-125 days +00:00:00,-126 days +23:59:37,1,1,cgw-platypus,0:0
2,30858137,COMPLETED,2021-08-06 19:04:39,2021-09-05 19:04:53,204800Mn,57553.77M,-125 days +00:00:00,-126 days +23:59:46,1,32,cgw-tbi01,0:0
3,30935078,COMPLETED,2021-08-09 16:52:51,2021-09-07 20:52:55,65536Mn,20577.96M,-121 days +00:00:00,-122 days +23:59:56,1,8,cgw-platypus,0:0
4,31364111_2,COMPLETED,2021-08-17 07:45:07,2021-09-10 16:45:24,16384Mn,9733.43M,-101 days +15:00:00,-101 days +14:59:43,1,1,production,0:0
5,31364111_3,COMPLETED,2021-08-17 07:45:07,2021-09-06 16:17:34,16384Mn,9708.04M,-101 days +15:00:00,-84 days +07:27:33,1,1,production,0:0


The USEDMEM column is the amount of memory used in MB per node.

The requested time (REQTIME) and used time (USEDTIME) columns are in d-hh:mm:ss or hh:mm:ss for jobs less than one day in duration.

**NODES** is the number of servers used for the job. Most jobs are single node. For multiple node jobs, memory usage is the maximum over all nodes.

**CPUS** is the total number of CPU cores allocated to the job, and for multi-node jobs, this includes all nodes.

Most jobs are run in the "production" or "nogpfs" partition. The "debug" and "sam" partitions are test jobs that are expected to be short, and the "maxwell", "pascal", and "turing" partitions are for GPU resources.

In [8]:
jobs['PARTITION'].value_counts()

PARTITION
production              7019136
nogpfs                   147044
pascal                   124451
sam                       64967
turing                    21412
maxwell                   11278
cgw-maizie                 4308
debug                      1616
cgw-platypus                376
cgw-dsi-gw                  228
cgw-capra1                  153
cgw-dougherty1              125
cgw-horus                    61
cgw-cqs1                     28
cgw-hanuman                  21
cgw-sideshowbob              14
cgw-vm-qa-flatearth1          9
cgw-tbi01                     7
Name: count, dtype: int64

The **EXITCODE** gives the [exit code](https://www.agileconnection.com/article/overview-linux-exit-codes) for the job, with "0:0" indicating a successful job. Exit codes have two numbers, where if the first number is non-zero, it indicates a problem on the server side and if the second is nonzero, it indicates a problem on the user side.

In [9]:
jobs['EXITCODE'].value_counts()

EXITCODE
0:0      7383829
1:0         4958
0:15        1887
0:125       1739
0:9         1361
2:0          508
0:7          389
121:0         89
127:0         88
13:0          68
24:0          67
0:11          35
38:0          32
28:0          29
6:0           27
126:0         24
0:6           18
0:2           16
7:0           12
29:0          12
16:0           9
59:0           8
9:0            4
0:40           4
8:0            3
125:0          3
0:105          2
76:0           1
85:0           1
0:12           1
30:0           1
0:98           1
43:0           1
3:0            1
67:0           1
4:0            1
11:0           1
0:8            1
103:0          1
116:0          1
Name: count, dtype: int64

In [10]:
jobs[jobs['EXITCODE'] == '1:0']

,JOBID,STATE,BEGIN,END,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE
18,31418105,NODE_FAIL,2021-08-19 10:09:50,2021-09-17 08:45:10,92160Mn,0,-172 days +12:00:00,-118 days +09:24:40,1,8,cgw-dougherty1,1:0
31996,31934490,FAILED,2021-09-01 09:08:52,2021-09-01 09:24:08,92160Mn,65881.35M,-13 days +04:00:00,0 days 00:15:16,1,1,maxwell,1:0
32199,31934755,FAILED,2021-09-01 09:35:01,2021-09-01 15:37:14,20000Mn,13323.77M,-5 days +12:00:00,0 days 06:02:13,1,1,production,1:0
32204,31934760,FAILED,2021-09-01 09:35:02,2021-09-01 13:07:26,20000Mn,10697.71M,-5 days +12:00:00,0 days 03:32:24,1,1,production,1:0
32205,31934762,FAILED,2021-09-01 09:35:04,2021-09-01 16:37:19,20000Mn,20336.22M,-5 days +12:00:00,0 days 07:02:15,1,1,production,1:0
...,...,...,...,...,...,...,...,...,...,...,...,...
7381905,25455341,FAILED,2020-10-29 23:21:16,2020-10-31 05:21:14,10240Mc,27792.37M,-5 days +14:00:00,-5 days +14:00:02,29,4,production,1:0
7381940,25455788,FAILED,2020-10-30 00:26:40,2020-10-30 20:26:43,5120Mc,28869.91M,0 days 20:00:00,0 days 20:00:03,22,4,production,1:0
7387598,25469985_10,FAILED,2020-10-30 13:46:33,2020-10-30 13:46:54,8192Mn,0,-5 days +20:00:00,0 days 00:00:21,1,4,production,1:0
7387599,25469985_11,FAILED,2020-10-30 13:46:26,2020-10-30 13:46:37,8192Mn,11.43M,-5 days +20:00:00,0 days 00:00:11,1,4,production,1:0


In [11]:
jobs[jobs['EXITCODE'] == '0:15']

,JOBID,STATE,BEGIN,END,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE
42,31669402,CANCELLED,2021-08-28 10:53:59,2021-09-05 10:53:57,65536Mn,5229.75M,-34 days +16:00:00,-31 days +20:00:02,9,10,production,0:15
1023,31798622,CANCELLED,2021-08-31 02:46:09,2021-09-05 02:46:09,40960Mn,1440.95M,-21 days +04:00:00,-21 days +04:00:00,4,4,turing,0:15
1029,31798672,CANCELLED,2021-08-31 02:53:48,2021-09-05 02:54:09,40960Mn,1438.46M,-21 days +04:00:00,-21 days +03:59:39,4,4,turing,0:15
32185,31934719,CANCELLED,2021-09-01 09:31:02,2021-09-01 09:31:07,20000Mn,0,-5 days +12:00:00,0 days 00:00:05,1,1,production,0:15
36103,31940094,CANCELLED,2021-09-01 14:55:55,2021-09-09 14:56:17,65536Mn,5054.56M,-34 days +16:00:00,-34 days +15:59:38,9,10,production,0:15
...,...,...,...,...,...,...,...,...,...,...,...,...
7387593,25469985_5,CANCELLED,2020-10-30 13:45:59,2020-10-31 13:46:06,8192Mn,4969.53M,-5 days +20:00:00,-5 days +19:59:53,1,4,production,0:15
7387594,25469985_6,CANCELLED,2020-10-30 13:45:57,2020-10-31 13:46:06,8192Mn,4976.77M,-5 days +20:00:00,-5 days +19:59:51,1,4,production,0:15
7387595,25469985_7,CANCELLED,2020-10-30 13:45:59,2020-10-31 13:46:06,8192Mn,4954.54M,-5 days +20:00:00,-5 days +19:59:53,1,4,production,0:15
7387596,25469985_8,CANCELLED,2020-10-30 13:45:59,2020-10-31 13:46:06,8192Mn,4951.88M,-5 days +20:00:00,-5 days +19:59:53,1,4,production,0:15


In [12]:
def df_to_datelist(df1):
    
    df = df1.copy(deep = True)
    df[3] = df[3].str.replace('time ', '')
    df[3] = df[3].astype(float)


    df = df[df[1] == 'user 9204']
    df = df[df[3] >= 15]
    df = df[df[4] == "returncode 1"]
    
    df['sbatch'] = df[5].apply(lambda x: 1 if 'sbatch' in x else 0)
    df = df[df['sbatch'] == 1]
    df[0] = pd.to_datetime(df[0])
    
    
    return df[0].to_list()

The slurm_wrapper_ce5.log and slurm_wrapper_ce6.log files contain logs of jobs submitted from the Open Science Grid.

In [13]:
"""
chunksize = 10 ** 6
for chunk in pd.read_csv(filename, chunksize=chunksize):
# chunk is a DataFrame. To "process" the rows in the chunk:
for index, row in chunk.iterrows():
print(row)
"""


'\nchunksize = 10 ** 6\nfor chunk in pd.read_csv(filename, chunksize=chunksize):\n# chunk is a DataFrame. To "process" the rows in the chunk:\nfor index, row in chunk.iterrows():\nprint(row)\n'

In [14]:
%%time
ce5 = pd.read_csv('../data/slurm_wrapper_ce5.log',
                  header=None,
                  delimiter=' - ',
                  engine='python')

ce6 = pd.read_csv('../data/slurm_wrapper_ce6.log',
                  header=None,
                  delimiter=' - ',
                  engine='python')

ce5.head()

errors_ce6 = df_to_datelist(ce6)
errors_ce5 = df_to_datelist(ce5)
all_errors = errors_ce5 + errors_ce6


CPU times: total: 43.3 s
Wall time: 1min 28s


In [15]:
all_errors[0]

Timestamp('2020-10-18 06:53:44.272915')

For this project, we are interested in jobs from user 9204 (the test user) where the command starts with '/usr/bin/squeue', the returncode is non-zero and the time is greater than 15. These conditions indicate that the scheduler becaem unresponsive at that point in time.

In [16]:
jobs['END'].value_counts()



END
2020-12-18 09:41:23    559
2021-10-02 22:48:56    551
2021-07-12 11:36:02    312
2021-10-03 13:36:05    311
2021-02-17 16:45:58    297
                      ... 
2021-06-26 12:51:45      1
2021-06-26 12:52:59      1
2021-06-26 12:53:50      1
2021-06-26 12:53:54      1
2020-10-31 23:49:43      1
Name: count, Length: 4109550, dtype: int64

In [17]:
jobs.loc[1]['END'] - jobs.loc[1]['BEGIN']

Timedelta('30 days 00:00:23')

In [18]:
jobs.loc[1]['END'] - jobs.loc[1]['REQTIME']

Timestamp('2022-01-08 11:36:32')

In [19]:
jobs.loc[1]['END'] - all_errors[1]

Timedelta('322 days 04:42:27.677588')

In [20]:
all_errors

[Timestamp('2020-10-18 06:53:44.272915'),
 Timestamp('2020-10-18 06:54:04.322412'),
 Timestamp('2020-10-18 07:47:25.825172'),
 Timestamp('2020-10-18 07:47:45.871008'),
 Timestamp('2020-10-18 07:53:33.972840'),
 Timestamp('2020-10-18 16:02:01.338468'),
 Timestamp('2020-10-18 20:52:15.737852'),
 Timestamp('2020-10-19 00:23:37.945125'),
 Timestamp('2020-10-19 00:23:57.979047'),
 Timestamp('2020-10-19 01:01:02.211847'),
 Timestamp('2020-10-19 01:01:22.392363'),
 Timestamp('2020-10-19 01:23:30.029296'),
 Timestamp('2020-10-19 01:23:50.057180'),
 Timestamp('2020-10-19 02:49:45.479887'),
 Timestamp('2020-10-19 03:38:35.181625'),
 Timestamp('2020-10-19 03:59:19.595905'),
 Timestamp('2020-10-19 17:54:52.404285'),
 Timestamp('2020-10-19 21:46:47.528895'),
 Timestamp('2020-10-20 00:18:40.214164'),
 Timestamp('2020-10-22 21:19:15.358639'),
 Timestamp('2020-10-28 14:24:14.349391'),
 Timestamp('2020-10-29 09:39:11.896357'),
 Timestamp('2020-10-29 09:39:32.287111'),
 Timestamp('2020-11-01 12:04:57.38

In [53]:
def count_jobs_before_interr(typeTime = 'h', countTime = 1, on = 'END'):
    """
    Calculates the number of jobs occurring within a specified time window 
    relative to each error timestamp, based on the relationship specified 
    (BEGIN, DURING, or END). Returns a DataFrame where each row corresponds 
    to an error and the number of jobs meeting the specified criteria.

    Parameters:
    ----------
    typeTime : str, optional
        The unit of time for the countTime parameter. Accepted values are:
        - 'm': Minutes
        - 'h': Hours (default)
        - 'd': Days

    countTime : float, optional
        The size of the time window in the units specified by typeTime. 
        For example:
        - countTime=1 with typeTime='h' means a 1-hour window.
        - countTime=30 with typeTime='m' means a 30-minute window.

    on : str, optional
        Defines the relationship between the jobs and the error timestamp. 
        Accepted values are:
        - 'BEGIN': Count jobs whose start times fall within the time window 
                   before the error.
        - 'DURING': Count jobs that were active (spanning) during the error.
        - 'END': Count jobs whose end times fall within the time window 
                 before the error. (Default)

    Returns:
    -------
    pd.DataFrame
        A DataFrame where each row corresponds to an error and its associated 
        count of jobs based on the specified criteria.
    """
    
    
    time_dict = {
        'm': 60,
        'h': 1,
        'd': 1/24    
    }
    
    
    if on == 'BEGIN':
        time_hours = countTime / time_dict[typeTime]
        job_counts_for_interrupt = {}
        jobs_copy = jobs.copy(deep = True)
        for i, error in enumerate(tqdm(all_errors, desc="Processing Errors")):
            hour_less_than_given = error - pd.Timedelta(hours=time_hours)
            count = ((jobs_copy['BEGIN'] > hour_less_than_given) & (jobs_copy['BEGIN'] <= error)).sum()
            job_counts_for_interrupt[error] = count
    
    elif on == 'DURING':
        time_hours = countTime / time_dict[typeTime]
        job_counts_for_interrupt = {}
        jobs_copy = jobs.copy(deep = True)
        for i, error in enumerate(tqdm(all_errors, desc="Processing Errors")):
            #hour_less_than_given = error - pd.Timedelta(hours=time_hours)
            count = ((jobs_copy['END'] > error) & (jobs_copy['BEGIN'] < error)).sum()
            job_counts_for_interrupt[error] = count
            
    elif on == 'END':
        time_hours = countTime / time_dict[typeTime]
        job_counts_for_interrupt = {}
        jobs_copy = jobs.copy(deep = True)
        for i, error in enumerate(tqdm(all_errors, desc="Processing Errors")):
            hour_less_than_given = error - pd.Timedelta(hours=time_hours)
            count = ((jobs_copy['END'] > hour_less_than_given) & (jobs_copy['END'] <= error)).sum()
            job_counts_for_interrupt[error] = count
            
    else:
        print(f'Your "ON" variable of "{on}" was not found to be (BEGIN, END, or DURING), so defaulted to END.')
        time_hours = countTime / time_dict[typeTime]
        job_counts_for_interrupt = {}
        jobs_copy = jobs.copy(deep = True)
        for i, error in enumerate(tqdm(all_errors, desc="Processing Errors")):
            hour_less_than_given = error - pd.Timedelta(hours=time_hours)
            count = ((jobs_copy['BEGIN'] > hour_less_than_given) & (jobs_copy['BEGIN'] <= error)).sum()
            job_counts_for_interrupt[error] = count

        
    
    
    return pd.DataFrame(job_counts_for_interrupt.items())

job_counts_for_interrupt

    

{Timestamp('2020-10-18 06:53:44.272915'): 444,
 Timestamp('2020-10-18 06:54:04.322412'): 445,
 Timestamp('2020-10-18 07:47:25.825172'): 396,
 Timestamp('2020-10-18 07:47:45.871008'): 397,
 Timestamp('2020-10-18 07:53:33.972840'): 352,
 Timestamp('2020-10-18 16:02:01.338468'): 934,
 Timestamp('2020-10-18 20:52:15.737852'): 765,
 Timestamp('2020-10-19 00:23:37.945125'): 763,
 Timestamp('2020-10-19 00:23:57.979047'): 763,
 Timestamp('2020-10-19 01:01:02.211847'): 925,
 Timestamp('2020-10-19 01:01:22.392363'): 921,
 Timestamp('2020-10-19 01:23:30.029296'): 1084,
 Timestamp('2020-10-19 01:23:50.057180'): 1088,
 Timestamp('2020-10-19 02:49:45.479887'): 1152,
 Timestamp('2020-10-19 03:38:35.181625'): 1279,
 Timestamp('2020-10-19 03:59:19.595905'): 1274,
 Timestamp('2020-10-19 17:54:52.404285'): 1683,
 Timestamp('2020-10-19 21:46:47.528895'): 2138,
 Timestamp('2020-10-20 00:18:40.214164'): 1825,
 Timestamp('2020-10-22 21:19:15.358639'): 1628,
 Timestamp('2020-10-28 14:24:14.349391'): 4191,
 Ti

In [55]:
count_jobs_interruption = count_jobs_before_interr('h',1,'END')

Processing Errors:  17%|█▋        | 551/3296 [00:40<03:19, 13.77it/s]


KeyboardInterrupt: 

In [48]:
len(all_errors)

3296

In [23]:
jobs = jobs[jobs['STATE'] == 'COMPLETED']

In [26]:
jobs.iloc[0]['END'] - all_errors[0]

Timedelta('322 days 04:42:47.727085')